In [113]:

import talib
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

In [114]:
df = yf.download("BTC-USD", start='2021-01-30', end='2022-06-02', interval='1d').drop(columns=['Adj Close'])
df.index = pd.to_datetime(df.index)
df.columns = df.columns.str.lower()

[*********************100%%**********************]  1 of 1 completed


### Стратегия MACD
#### Если MACD > MACD_Signal, входим и держим позицию BUY
#### Если MACD < MACD_Signal, входим и держим позицию SELL


In [115]:
macd_df = df.copy()
macd_df['MACD'], macd_df['MACD_Signal'], macd_df['MACD_Hist'] = talib.MACD(macd_df["close"], fastperiod=12, slowperiod=26, signalperiod=9)

# Инициализация сигналов
macd_df['Signal'] = 0

# Логика сигналов
macd_df.loc[macd_df['MACD'] > macd_df['MACD_Signal'], 'Signal'] = 1
macd_df.loc[macd_df['MACD'] < macd_df['MACD_Signal'], 'Signal'] = -1


# Сигналы позиций
buy_signals = macd_df[macd_df['Signal'] == 1]
sell_signals = macd_df[macd_df['Signal'] == -1]

# Создание графика с помощью Plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.1, 
                    subplot_titles=('Price with Buy/Sell Signals', 'MACD'))

# Добавление линии цены закрытия
fig.add_trace(go.Scatter(x=macd_df.index, y=macd_df['close'], mode='lines', name='Close Price'), row=1, col=1)

# Добавление точек для Long сигналов
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', 
                         marker=dict(color='green', size=10), name='Buy Signal'), row=1, col=1)

# Добавление точек для Short сигналов
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', 
                         marker=dict(color='red', size=10), name='Sell Signal'), row=1, col=1)

# Добавление MACD линии
fig.add_trace(go.Scatter(x=df.index, y=macd_df['MACD'], mode='lines', name='MACD', line=dict(color='blue')), row=2, col=1)

# Добавление сигнальной линии MACD
fig.add_trace(go.Scatter(x=df.index, y=macd_df['MACD_Signal'], mode='lines', name='MACD Signal', line=dict(color='orange')), row=2, col=1)

# Настройки оформления графика
fig.update_layout(title='Price and MACD with Buy/Sell Signals',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis2_title='Date',
                  yaxis2_title='MACD',
                  legend=dict(x=1, y=1),
                  
)
# Показать график
fig.show()

### Стратегия STOCH
#### Если K > D, входим и держим позицию BUY
#### Если K < D, входим и держим позицию SELL

In [116]:

stoch_df = df.copy()
# Расчет стохастического осциллятора
stoch_df['slowk'], stoch_df['slowd'] = talib.STOCH(stoch_df['high'], stoch_df['low'], stoch_df['close'], 
                                        fastk_period=14, slowk_period=7, slowk_matype=0, 
                                        slowd_period=7, slowd_matype=0)

# Инициализация сигналов
stoch_df['Signal'] = 0


# Логика для генерации сигналов
stoch_df.loc[stoch_df['slowk'] > stoch_df['slowd'], 'Signal'] = 1
stoch_df.loc[stoch_df['slowk'] < stoch_df['slowd'], 'Signal'] = -1


# Получаем сигналы для визуализации
buy_signals = stoch_df[stoch_df['Signal'] == 1]
sell_signals = stoch_df[stoch_df['Signal'] == -1]

# Создание графика с помощью Plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.1, 
                    subplot_titles=('Price with Buy/Sell Signals', 'Stochastic Oscillator'))

# Добавление линии цены закрытия
fig.add_trace(go.Scatter(x=stoch_df.index, y=stoch_df['close'], mode='lines', name='Close Price'), row=1, col=1)

# Добавление точек для Long сигналов
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', 
                         marker=dict(color='green', size=10), name='Buy Signal'), row=1, col=1)

# Добавление точек для Short сигналов
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', 
                         marker=dict(color='red', size=10), name='Sell Signal'), row=1, col=1)


# Добавление Stochastic Oscillator
fig.add_trace(go.Scatter(x=stoch_df.index, y=stoch_df['slowk'], mode='lines', name='%K', line=dict(color='purple')), row=2, col=1)
fig.add_trace(go.Scatter(x=stoch_df.index, y=stoch_df['slowd'], mode='lines', name='%D', line=dict(color='orange')), row=2, col=1)

# Настройки оформления графика
fig.update_layout(title='Price and Stochastic Oscillator with Buy/Sell Signals',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis2_title='Date',
                  yaxis2_title='Stochastic',
                  legend=dict(x=1, y=1),
                  
)

# Показать график
fig.show()

### Стратегия TEMA
#### Если цена > TripleEMA, входим и держим позицию BUY
#### Если цена < TripleEMA, входим и держим позицию SELL

In [96]:
tema_df = df.copy()
period = 55
# EMA1: Первоначальная EMA от цены
tema_df['EMA1'] = talib.EMA(tema_df['close'], timeperiod=period)
    
    # EMA2: EMA от EMA1
tema_df['EMA2'] = talib.EMA(tema_df['EMA1'], timeperiod=period)
    
    # EMA3: EMA от EMA2
tema_df['EMA3'] = talib.EMA(tema_df['EMA2'], timeperiod=period)
    
    # Triple EMA
tema_df['TripleEMA'] = (3 * tema_df['EMA1']) - (3 * tema_df['EMA2']) + tema_df['EMA3']

tema_df = tema_df.dropna()

# Инициализация сигналов
tema_df['Signal'] = 0

# Логика для входа в короткую позицию (Short)
tema_df.loc[tema_df['close'] < tema_df['TripleEMA'], 'Signal'] = -1

# Логика для входа в длинную позицию (Long)
tema_df.loc[tema_df['close'] > tema_df['TripleEMA'], 'Signal'] = 1

buy_signals = tema_df[tema_df['Signal'] == 1]
sell_signals = tema_df[tema_df['Signal'] == -1]

# Визуализация сигналов

fig = go.Figure()
fig.add_trace(go.Scatter(x=tema_df.index, y=tema_df['close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=tema_df.index, y=tema_df['TripleEMA'], mode='lines', name='Triple EMA', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))
fig.update_layout(title='Triple EMA Strategy', xaxis_title='Date', yaxis_title='Price')
fig.show()



### Стратегия с использованием линий Болинджера
#### Если цена > UpperBB_1SD, входим и держим позицию BUY
#### Если цена < LowerBB_1SD, входим и держим позицию SELL
#### Выход из позиции BUY, если цена < UpperBB_1SD
#### Выход из позиции SELL, если цена > LowerBB_1SD

In [110]:
bb_df = df.copy()

# Вычисляем среднюю линию (SMA)
bb_df['MA'] = talib.SMA(bb_df['close'], timeperiod=period)
    
# Вычисляем стандартное отклонение
bb_df['STD'] = bb_df['close'].rolling(window=period).std()
    
# Вычисляем верхние и нижние полосы для разных уровней стандартного отклонения
bb_df['UpperBB_1SD'] = bb_df['MA'] + (1 * bb_df['STD'])
bb_df['UpperBB_2SD'] = bb_df['MA'] + (2 * bb_df['STD'])
bb_df['LowerBB_1SD'] = bb_df['MA'] - (1 * bb_df['STD'])
bb_df['LowerBB_2SD'] = bb_df['MA'] - (2 * bb_df['STD'])


bb_df['Signal'] = 0 # Инициализация сигналов
bb_df['Exit'] = 0    # Инициализация выхода

# Логика для определения сигнала
bb_df['Signal'] = 0  # Инициализируем столбец Signal

# Вход в позицию BUY
bb_df.loc[bb_df['close'] > bb_df['UpperBB_1SD'], 'Signal'] = 1
# Вход в позицию SELL
bb_df.loc[bb_df['close'] < bb_df['LowerBB_1SD'], 'Signal'] = -1
# Логика выхода из позиции
bb_df['Exit'] = 0  # Инициализируем столбец Exit

bb_df.loc[(bb_df['Signal'].shift(1) == 1) & (bb_df['close'] < bb_df['UpperBB_1SD']), 'Exit'] = 2
bb_df.loc[(bb_df['Signal'].shift(1) == -1) & (bb_df['close'] > bb_df['LowerBB_1SD']), 'Exit'] = 2

buy_signals = bb_df[bb_df['Signal'] == 1]
sell_signals = bb_df[bb_df['Signal'] == -1]
exit_signals = bb_df[bb_df['Exit'] != 0]


fig = go.Figure()

fig.add_trace(go.Scatter(x=bb_df.index, y=bb_df['close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))
fig.add_trace(go.Scatter(x=exit_signals.index, y=exit_signals['close'], mode='markers', marker=dict(color='orange', size=10), name='Exit Signal'))
fig.add_trace(go.Scatter(x=bb_df.index, y=bb_df['UpperBB_1SD'], mode='lines', name='Upper Band 1SD', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=bb_df.index, y=bb_df['LowerBB_1SD'], mode='lines', name='Lower Band 1SD', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=bb_df.index, y=bb_df['UpperBB_2SD'], mode='lines', name='Upper Band 2SD', line=dict(color='red')))
fig.add_trace(go.Scatter(x=bb_df.index, y=bb_df['LowerBB_2SD'], mode='lines', name='Lower Band 2SD', line=dict(color='red')))
fig.update_layout(title='Bollinger Bands Strategy with Exits', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [112]:


# Стратегия паттернов свечей
candlestick_df = df.copy()

# Распознавание паттерна поглощения
candlestick_df['engulfing'] = talib.CDLENGULFING(candlestick_df['open'], candlestick_df['high'], candlestick_df['low'], candlestick_df['close'])

# Генерация сигналов
candlestick_df['signal'] = 0  # Инициализация столбца сигналов нулями

# Сигналы на покупку (бычье поглощение)
candlestick_df.loc[candlestick_df['engulfing'] > 0, 'signal'] = 1

# Сигналы на продажу (медвежье поглощение)
candlestick_df.loc[candlestick_df['engulfing'] < 0, 'signal'] = -1

# Создание графика с двумя подграфиками
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, 
                    subplot_titles=('Свечной график с сигналами', 'Значения паттерна поглощения'))

# Добавляем свечной график
fig.add_trace(go.Candlestick(x=candlestick_df.index,
                             open=candlestick_df['open'],
                             high=candlestick_df['high'],
                             low=candlestick_df['low'],
                             close=candlestick_df['close'],
                             name='Свечи'),
              row=1, col=1)

# Добавляем сигналы
buy_signals = candlestick_df[candlestick_df['signal'] == 1]
sell_signals = candlestick_df[candlestick_df['signal'] == -1]

fig.add_trace(go.Scatter(
    x=buy_signals.index, 
    y=buy_signals['low'] - (candlestick_df['high'] - candlestick_df['low']).mean(),
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Бычье поглощение'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=sell_signals.index, 
    y=sell_signals['high'] + (candlestick_df['high'] - candlestick_df['low']).mean(),
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Медвежье поглощение'
), row=1, col=1)

# Добавляем линию engulfing на второй подграфик
fig.add_trace(go.Scatter(
    x=candlestick_df.index,
    y=candlestick_df['engulfing'],
    mode='lines',
    name='Значения паттерна поглощения'
), row=2, col=1)

# Добавляем горизонтальные линии для значений 100 и -100
fig.add_shape(type="line", x0=candlestick_df.index[0], x1=candlestick_df.index[-1], y0=100, y1=100,
              line=dict(color="green", width=1, dash="dash"), row=2, col=1)
fig.add_shape(type="line", x0=candlestick_df.index[0], x1=candlestick_df.index[-1], y0=-100, y1=-100,
              line=dict(color="red", width=1, dash="dash"), row=2, col=1)

fig.update_layout(
    title='Стратегия: Паттерны свечей (Поглощение)',
    xaxis_title='Дата',
    yaxis_title='Цена',
    xaxis2_title='Дата',
    yaxis2_title='Значение паттерна',
    legend=dict(x=1, y=1),
    xaxis_rangeslider_visible=False
)

fig.show()

